In [4]:
## imports
# astronomy
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
# plotting
import matplotlib.pyplot as plt
# data 
import numpy as np
import pandas as pd 

# 1. Read Data

In [5]:
# file paths 
path_catalogs = 'C:\\Users\\polar\\OneDrive - The University of Kansas\\AGNerds\\Catalogs'

In [13]:
# open tphot
tphot_data = ascii.read(path_catalogs+'\\tphot.cat')
tphot_cols = tphot_data.colnames

# show table
tphot_data.show_in_notebook()
print(tphot_cols)

# important columns:
# name = 'ra'
# name = 'dec'

['field', 'ra', 'dec', 'f560w_uJy', 'f560w_uJy_err', 'f770w_uJy', 'f770w_uJy_err', 'f1000w_uJy', 'f1000w_uJy_err', 'f1280w_uJy', 'f1280w_uJy_err', 'f1500w_uJy', 'f1500w_uJy_err', 'f1800w_uJy', 'f1800w_uJy_err', 'f2100w_uJy', 'f2100w_uJy_err']


In [11]:
# open EGS F22
egs_f22_data = Table.read(path_catalogs+'\\EGS_F22.fits.gz')
egs_f22_cols = egs_f22_data.columns

# print all columns
for col in egs_f22_cols:
    print(col)

ID
RA
DEC
CANDELS_RA
CANDELS_DEC
F606W
F814W
F105W
F125W
F140W
F160W
F36
F45
DF606W
DF814W
DF105W
DF125W
DF140W
DF160W
DF36
DF45
ZA
ZL68
ZU68
ZPEAK
ZA_NOIRAC
ZL68_NOIRAC
ZU68_NOIRAC
ZPEAK_NOIRAC


In [12]:
# open EGS FIR
egs_fir_data = Table.read(path_catalogs+'\\egs_FIR_photometry_catalog.fits')
egs_fir_cols = egs_f22_data.columns

# print all columns
for col in egs_fir_cols:
    print(col)

ID
RA
DEC
CANDELS_RA
CANDELS_DEC
F606W
F814W
F105W
F125W
F140W
F160W
F36
F45
DF606W
DF814W
DF105W
DF125W
DF140W
DF160W
DF36
DF45
ZA
ZL68
ZU68
ZPEAK
ZA_NOIRAC
ZL68_NOIRAC
ZU68_NOIRAC
ZPEAK_NOIRAC
